# Mini-ChatGPT

Special thanks to Leandro von Werra 's https://github.com/lvwerra/trl repo

diagram from https://openai.com/blog/chatgpt/

<img src='https://cdn.openai.com/chatgpt/draft-20221129c/ChatGPT_Diagram.svg'>

#### Figure 1

### Step 1

Supervised Fine-Tuning (SFT) is the process of taking a model already pre-trained on a very large corpus of data, and re-training it carefully as not to erase the knowledge the model has gained from that pre-training [catastropic forgetting](https://github.com/clam004/intro_continual_learning), but rather slightly nudge the model to perform better in a more specific or narrow domain represented by a much smaller fine tuning dataset.

There are already plenty of resources on Supervised Fine-Tuning (SFT) already, some mistakenly claiming to help you build a mini-chatgpt. I will not cover SFT, instead I will point you to some of my favorite resources. 

Here is an explanatory video by Huggingface of the Decoder, or autoregressive, transformers: https://youtu.be/d_ixlCubqQw

Here is a great blog explaining language modeling and the cross entropy loss function https://lena-voita.github.io/nlp_course/language_modeling.html 

Fine-tuning GPT-like models: https://huggingface.co/course/chapter7/6?fw=pt 

ChatGPT likely was fine-tuned on a dataset of instructions and examples of following to
 
### Step 2

Fine-tuning usually means training for only a small number of epochs with a much smaller learning rate, but also using the cross entropy loss function used to do pre-training, but this way to learning is abit limited, can you tell why? 

<img src="https://lena-voita.github.io/resources/lectures/lang_models/neural/one_step_loss_intuition-min.png">

#### Figure 2

This example of using cross entropy shows how the training algorithm rewards or penalizes based on how large the logit (probability mass asigned to) for "cat" is, just this one way of completing the sequence, but we know that in reality when generating responses to instructions, there isnt one right ways to do it even in the cases when there is one right answer, there are many ways to be good and many ways to be bad. 

For the sake of keeping this tutorial light weight and intuitive, the reward modeling has been simplified as compared to InstructGPT https://openai.com/blog/instruction-following/ which is the model that ChatGPT is a scaled up and improved version of. 

Instead of training a reward model based on human rankings of output text, which was done to make the reward signal more stable or reliable, we use the more direct tactic of using the logits from a classifier as the reward signal, "positive meanss do more like this, negative means do less like this". 

### Step 3

Given our limited time, I am going to focus on giving you what is hard to find. That is, good explainations of the parts that are usually glossed over yet important, or those parts that are usually explained in a much more jargony, field specific, technical or mathematical manner.

That means we will be focusing on step 3 in Figure 1. Namely, how reinforcement learning, something we have seen thus far in the popular media mostly applied to computer games, instead applied to natural language, or the generation of sequences of discrete tokens. 

In [1]:
%load_ext autoreload
%autoreload 2

# this version of the notebook requires weights and biases
# !pip install wandb

In [2]:
from minichatgpt.experiments.imdb import imdb_config, imdb_sent_kwargs
from minichatgpt.processdata.build_dataset import build_dataset
from minichatgpt import PPOTrainer, LengthSampler, Lab

NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
lab = Lab(imdb_config)

dataset = lab.build_dataset(dataset_name="imdb",input_min_text_length=2,input_max_text_length=8)
new_policy, old_policy, tokenizer = lab.init_policies_tokenizer()
ppo_trainer = lab.init_ppo_trainer(imdb_config, new_policy, old_policy, tokenizer, dataset)
reward_model = lab.init_reward_model()

Found cached dataset imdb (/Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Loading cached processed dataset at /Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-2bd6a5d7d39a840d.arrow
Loading cached processed dataset at /Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-2ecf25d24c93f132.arrow
wandb: Currently logged in as: carsonlam004. Use `wandb login --relogin` to force relogin


In [4]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch['input_ids']
    break

NameError: name 'tqdm' is not defined